In [2]:
import requests
import pandas as pd
import numpy as np
# states full names

dic_state_name = {'AP': 'Andhra Pradesh',
 'AR': 'Arunachal Pradesh',
 'AS': 'Assam',
 'BR': 'Bihar',
 'CT': 'Chhattisgarh',
 'GA': 'Goa',
 'GJ': 'Gujarat',
 'HR': 'Haryana',
 'HP': 'Himachal Pradesh',
 'JH': 'Jharkhand',
 'KA': 'Karnataka',
 'KL': 'Kerala',
 'MP': 'Madhya Pradesh',
 'MH': 'Maharashtra',
 'MN': 'Manipur',
 'ML': 'Meghalaya',
 'MZ': 'Mizoram',
 'NL': 'Nagaland',
 'OR': 'Odisha',
 'PB': 'Punjab',
 'RJ': 'Rajasthan',
 'SK': 'Sikkim',
 'TN': 'Tamil Nadu',
 'TG': 'Telangana',
 'TR': 'Tripura',
 'UP': 'Uttar Pradesh',
 'UT': 'Uttarakhand',
 'WB': 'West Bengal',
 'AN': 'Andaman and Nicobar Islands',
 'CH': 'Chandigarh',
 'DN': 'Dadra and Nagar Haveli and Daman and Diu',
 'DL': 'Delhi',
 'JK': 'Jammu and Kashmir',
 'LA': 'Ladakh',
 'LD': 'Lakshadweep',
 'PY': 'Puducherry'}
state_data = requests.get('https://data.covid19india.org/v4/min/data.min.json')
states = state_data.json()

In [3]:
list2 = []
for state in states.keys():
    if state == 'TT':
        continue
    else:
        d1 = states[state]["districts"]
        for district in d1.keys():
            population = d1[district]['meta'].get('population', 0) if 'meta' in d1[district] else 0
            total = d1[district].get('total', {})
            confirmed = total.get('confirmed', 0)
            deceased = total.get('deceased', 0)
            recovered = total.get('recovered', 0)
            tested = total.get('tested', 0)
            vaccinated1 = total.get('vaccinated1', 0)
            vaccinated2 = total.get('vaccinated2', 0)

            
            if population > 0:
                testing_ratio = tested / population
            else:
                testing_ratio = 0

            
            if 0.05 <= testing_ratio <= 0.1:
                category = 'A'
            elif 0.1 < testing_ratio <= 0.3:
                category = 'B'
            elif 0.3 < testing_ratio <= 0.5:
                category = 'C'
            elif 0.5 < testing_ratio <= 0.75:
                category = 'D'
            elif 0.75 < testing_ratio <= 1.0:
                category = 'E'
            else:
                category = 'Uncategorized'

            dic = {
                'state_name': dic_state_name[state],
                'State': state,
                'District': district,
                'Population': population,
                'Confirmed': confirmed,
                'Deceased': deceased,
                'Recovered': recovered,
                'Tested': tested,
                'Vaccinated1': vaccinated1,
                'Vaccinated2': vaccinated2,
                'TestingRatio': testing_ratio,
                'Category': category
            }
            list2.append(dic)

df = pd.DataFrame(list2)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


print(df.head(10))

                    state_name State                  District  Population  \
0  Andaman and Nicobar Islands    AN                  Nicobars       36842   
1  Andaman and Nicobar Islands    AN  North and Middle Andaman      105597   
2  Andaman and Nicobar Islands    AN             South Andaman      238142   
3  Andaman and Nicobar Islands    AN                   Unknown           0   
4               Andhra Pradesh    AP                 Anantapur     4083315   
5               Andhra Pradesh    AP                  Chittoor     4170468   
6               Andhra Pradesh    AP             East Godavari     5151549   
7               Andhra Pradesh    AP          Foreign Evacuees           0   
8               Andhra Pradesh    AP                    Guntur     4889230   
9               Andhra Pradesh    AP                   Krishna     4529009   

   Confirmed  Deceased  Recovered  Tested  Vaccinated1  Vaccinated2  \
0          0         0          0       0        25394        20313   

In [4]:
import pandas as pd
df['Deceased'] = pd.to_numeric(df['Deceased'], errors='coerce')
death_analysis = df.groupby('Category')['Deceased'].sum().reset_index()
print(death_analysis)

        Category  Deceased
0              A     52401
1              B    148011
2              C     21024
3              D     16281
4              E       820
5  Uncategorized    219933
